<a href="https://colab.research.google.com/github/tamyriscabral/ED2/blob/main/Trabalho_IC_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

url = '/content/cities.csv'

cidades = pd.read_csv(url)

FileNotFoundError: [Errno 2] No such file or directory: '/content/cities.csv'

In [ ]:
def load_cities(csv_file):
    """
    Load cities from a CSV file
    """
    cities_df = pd.read_csv(csv_file)
    print(f"Loaded {len(cities_df)} cities from {csv_file}")
    return cities_df

In [ ]:
def plot_cities(cities_df=None, csv_file=None, show_indices=True, save_plot=False, filename='cities_plot.png'):
    """
    Plot the cities
    """
    # Load cities from CSV if not provided as DataFrame
    if cities_df is None and csv_file is not None:
        cities_df = load_cities(csv_file)
    elif cities_df is None:
        raise ValueError("Either cities_df or csv_file must be provided")

    # Get distance unit from the DataFrame attributes
    distance_unit = cities_df.attrs.get('distance_unit', 'units')

    plt.figure(figsize=(10, 8))
    plt.scatter(cities_df['x'], cities_df['y'], c='blue', s=100)

    if show_indices:
        for i, row in cities_df.iterrows():
            plt.annotate(str(row['city_id']),
                         (row['x'] + 0.5, row['y'] + 0.5),
                         fontsize=12)

    plt.title('Cities for TSP')
    plt.xlabel(f'X Coordinate ({distance_unit})')
    plt.ylabel(f'Y Coordinate ({distance_unit})')
    plt.grid(True)

    if save_plot:
        plt.savefig(filename)
        print(f"Plot saved to {filename}")

    plt.show()

In [ ]:
def plot_paths(cities_df, paths, show_indices=True, save_plot=False, filename='paths_plot.png',
               base_coords=None, return_to_base=False):
    """
    Plot multiple paths between cities

    Args:
        cities_df (DataFrame): DataFrame containing city information with 'city_id', 'x', and 'y' columns
        paths (list): List of lists, where each inner list contains a sequence of city_ids representing a path
        show_indices (bool): Whether to show city indices on the plot
        save_plot (bool): Whether to save the plot to a file
        filename (str): Filename to save the plot if save_plot is True
        base_coords (tuple): Coordinates of the base location (x, y) if return_to_base is True
        return_to_base (bool): Whether paths should return to the base location
    """
    # Get distance unit from the DataFrame attributes
    distance_unit = cities_df.attrs.get('distance_unit', 'units')

    plt.figure(figsize=(12, 10))

    # Plot all cities
    plt.scatter(cities_df['x'], cities_df['y'], c='gray', s=50, alpha=0.5)

    # Plot base location if return_to_base is True
    if return_to_base and base_coords is not None:
        plt.scatter(base_coords[0], base_coords[1], c='black', s=100, marker='*', label='Base')

    # Colors for different paths
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'pink', 'cyan', 'magenta', 'yellow']

    # Plot each path
    for i, path in enumerate(paths):
        path_color = colors[i % len(colors)]

        # Get coordinates for the path
        x_coords = cities_df.loc[path, 'x'].values
        y_coords = cities_df.loc[path, 'y'].values

        # If returning to base, add base coordinates to start and end of path
        if return_to_base and base_coords is not None and len(path) > 0:
            x_coords = np.concatenate(([base_coords[0]], x_coords, [base_coords[0]]))
            y_coords = np.concatenate(([base_coords[1]], y_coords, [base_coords[1]]))

        # Calculate path distance
        path_distance = 0
        for j in range(len(x_coords) - 1):
            dx = x_coords[j+1] - x_coords[j]
            dy = y_coords[j+1] - y_coords[j]
            path_distance += np.sqrt(dx**2 + dy**2)

        # Plot the path
        plt.plot(x_coords, y_coords, 'o-', color=path_color, linewidth=2,
                 label=f'Path {i+1}: {len(path)} cities, {path_distance:.2f} {distance_unit}')

        # Add city labels if requested
        if show_indices:
            for j, city_id in enumerate(path):
                plt.annotate(str(city_id), (cities_df.loc[city_id, 'x'], cities_df.loc[city_id, 'y']), fontsize=10)

    plt.title('Travel Paths Between Cities')
    plt.xlabel(f'X Coordinate ({distance_unit})')
    plt.ylabel(f'Y Coordinate ({distance_unit})')
    plt.legend()
    plt.grid(True)

    if save_plot:
        plt.savefig(filename)
        print(f"Plot saved to {filename}")

    plt.show()


In [ ]:
#Função para calcular distâncias entre duas cidades
def distancia_cidades(cidade_1, cidade_2):
  return np.sqrt((cidade_1['x']-cidade_2['x'])**2 + (cidade_1['y']-cidade_2['y'])**2  )


In [ ]:
def avaliacao_rota(rota, cidades, D_diaria=60, max_dias=5):
    total_visitado = 1  # Conta a cidade de partida
    distancia_percorrida_dia = 0
    d_total = 0
    dias = 1

    for i in range(len(rota) - 1):
        cidade_origem = cidades.iloc[rota[i]]
        cidade_destino = cidades.iloc[rota[i + 1]]
        distancia = distancia_cidades(cidade_origem, cidade_destino)
        d_total += distancia

        if distancia_percorrida_dia + distancia <= D_diaria:
            distancia_percorrida_dia += distancia
            total_visitado += 1
        else:
            dias += 1
            if dias > max_dias:
                break
            if distancia <= D_diaria:
                distancia_percorrida_dia = distancia
                total_visitado += 1
            else:
                break

    return d_total  # Deve estar aqui, fora do loop


In [ ]:
def selecao_torneio(populacao, fitness, num_individuos=5):
  selecionado = random.sample(list(zip(populacao, fitness)), num_individuos)
  selecionado = sorted(selecionado, key=lambda x:x[1], reverse=True)
  return selecionado[0][0]

In [ ]:
def order_crossover(pai_1, pai_2):
  size = len(pai_1)
  filho = [None]*size
  a, b = sorted(random.sample(range(size),2))

  filho[a:b+1] = pai_1[a:b+1]

  posicoes_vazias = [i for i in range(size) if filho[i] == None]
  gene_pai_2 = [gene for gene in pai_2 if gene not in filho]

  for pos, gene in zip(posicoes_vazias, gene_pai_2):
    filho[pos] = gene
  return filho

In [ ]:
def mutacao(individuo):
  a, b = random.sample(range(len(individuo)), 2)
  individuo[a], individuo[b] = individuo[b], individuo[a]
  return individuo

In [ ]:
def algoritmo_genetico(cidades, tam_pop= 100, geracoes= 500, taxa_mutacao= 0.2):
  n_cidades = len(cidades)
  populacao = [random.sample(range(n_cidades), n_cidades) for _ in range(tam_pop)]

  melhor_rota = None
  melhor_fit = 100000000

  for gen in range(geracoes):
    fitness = [avaliacao_rota(ind, cidades) for ind in populacao]
    max_fit = min(fitness)
    if max_fit < melhor_fit:
      melhor_fit = max_fit
      melhor_rota = populacao[fitness.index(max_fit)]

    nova_pop = []
    rotas = []

    for _ in range(tam_pop):
      pai_1 = selecao_torneio(populacao, fitness)
      pai_2 = selecao_torneio(populacao, fitness)
      filho = order_crossover(pai_1, pai_2)

      if random.random() < taxa_mutacao:
        filho = mutacao(filho)
      nova_pop.append(filho)
    populacao = nova_pop

    if(gen + 1) % 50 == 0:
      print(f'Geração {gen+1}, melhor fitness: {melhor_fit}')
      rotas.append(melhor_rota)
  return melhor_rota, melhor_fit, rotas

In [ ]:
melhor_rota, fitness, rotas = algoritmo_genetico(cidades)

print(melhor_rota)
print(fitness)
print(rotas)